In [1]:
# imports
import os
import math
import dill
from itertools import permutations

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import gridspec
from sklearn.linear_model import LinearRegression as linreg

import suite2p as s2p

import TwoPUtils
import STX3KO_analyses as stx



%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
basedir_VR = '/home/mplitt/VR_scratch'
figdir_local = '/home/mplitt/Fig_scratch'
figdir_remote = '/STX3_2P_results/scratch'

In [33]:
mouse = '4467975.1'
pkldir = os.path.join('/home/mplitt/YMazeSessPkls/',mouse)

    
if mouse in stx.ymaze_sess_deets.KO_sessions.keys():  
    sessions_deets = stx.ymaze_sess_deets.KO_sessions_ravel[mouse]
elif mouse in stx.ymaze_sess_deets.CTRL_sessions.keys():
    sessions_deets = stx.ymaze_sess_deets.CTRL_sessions_ravel[mouse]
else:
    print( "mouse ID typo")



    
sess_list = []
for deets in sessions_deets:
    print(deets['date'],deets['scene'])
    
    with open(os.path.join(pkldir,deets['date'], "%s_%d.pkl" % (deets['scene'],deets['session'])),'rb') as file:
        sess_list.append(dill.load(file))
        
        

28_09_2020 YMaze_LNovel
28_09_2020 YMaze_LNovel
29_09_2020 YMaze_LNovel
30_09_2020 YMaze_LNovel
01_10_2020 YMaze_LNovel
02_10_2020 YMaze_LNovel
03_10_2020 YMaze_LNovel
04_10_2020 YMaze_LNovel
04_10_2020 YMaze_RewardReversal
05_10_2020 YMaze_RewardReversal
06_10_2020 YMaze_RewardReversal
06_10_2020 YMaze_LNovel


In [50]:
for sess in sess_list:
    reward_mask = sess.vr_data['reward']._values>0
    reward_start = np.argwhere(reward_mask).ravel()
    reward_end = (reward_start + int(2*sess.scan_info['frame_rate'])).astype(np.int)

    consum_mask = np.zeros(reward_mask.shape)>0
    for (start,end) in zip(reward_start,reward_end):
#         print(start,end)
        consum_mask[start:end]=True

    antic_licks = np.copy(sess.vr_data['lick']._values)
    antic_licks[consum_mask]=0
    
    sess.add_timeseries(antic_licks=antic_licks, speed = sess.vr_data['dz']._values,licks=sess.vr_data['lick']._values)
    sess.add_pos_binned_trial_matrix(('antic_licks','licks','speed'),'t',min_pos=6,max_pos=43,bin_size=1,mat_only=True)
    
    
    antic_lick_positions = np.zeros(sess.timeseries['licks'].shape)
    antic_lick_mask = sess.timeseries['antic_licks']>0
#     print(lick_mask.shape)
    antic_lick_positions[antic_lick_mask] = sess.vr_data['t']._values[antic_lick_mask.ravel()]
    sess.add_timeseries(antic_lick_positions=antic_lick_positions)
    
    # find probe trials (no reward delivered between trial start and stop and no licks in correct reward zone)

In [ ]:
def get_probes_and_omissions(sess):
    
    
    rzone_early = slice(25-6,32-6)
    rzone_late = slice(35-6,42-6)

In [52]:
sess = sess_list[0]
sess.trial_info

{'block_number': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 'LR': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.])}

In [ ]:
def single_trial_lick_metrics(sess):
    
    if ~reversal:
        rzone_left = {'xcenter': -31.6, 'zcenter': 181.6, 'scale': 25}
        rzone_right = {'xcenter': 88.2, 'zcenter': 238.2, 'scale': 25}
    else:
        rzone_left = {'xcenter': -88.7, 'zcenter': 238.7, 'scale': 25}
        rzone_right = {'xcenter': 31.6, 'zcenter': 181.6, 'scale': 25}
    
    
    
    
    for trial, (start,stop, lr) in zip(sess.trial_start_inds,sess.teleport_inds, sess.trial_info['LR']):
        
       
        # lick rate pre rzone 0
        
        # lick rate pre rzone 1
    
        # lick discrimination - (lr0-lr1)/(lr0+lr1)
        
        
        # lick accuracy - fraction of licks in correct reward zone plust 50 cm prior
        
        # lick position variance
        

In [ ]:
def licking_vectorlength(SessDict):
    
    vl_dict ={}
    for m,(mouse,days) in enumerate(SessDict.items()):
        print(mouse)    
#         LR, LICKS, SPEED = [],[],[]
#         transition_trials = []
        famarm_vl,novelarm_vl= [], []
        
        for i, day in enumerate(days):
                  
            for sess_ind,session in enumerate(day):
                sess = TwoPUtils.sess.Session(basedir_VR=basedir_VR,mouse = mouse, date = session['date'], scene= session['scene'], 
                                              session = session['session'], VR_only=True, prompt_for_keys=False )
                sess.align_VR_to_2P()

                # get LR value for each trial
                lr_trial = get_LR_trial(sess)

                # make position binned lick rates and speed
                sess.add_timeseries(licks=sess.vr_data['lick']._values,speed= sess.vr_data['dz']._values)
                sess.add_pos_binned_trial_matrix(('licks','speed'),'t',min_pos=6,max_pos=43,bin_size=1,mat_only=True)
                
                licks = np.copy(sess.trial_matrices['licks'])
                licks[np.isnan(licks)]=0.
                licks = sp.ndimage.filters.gaussian_filter1d(licks,1,axis=-1)
                licks+=1E-3
                licks=licks/licks.sum(axis=-1,keepdims=True)
                phase = np.linspace(0,1,num=licks.shape[1])[np.newaxis,:]
                mean = np.sum(licks*phase,axis=-1,keepdims=True)
                lick_vl = (licks*np.power(phase - mean,2)).sum(axis=-1)
                
#                 phase = np.linspace(0,2*np.pi,num=licks.shape[1])
#                 phase = np.array([complex(0,p) for p in phase.tolist()])
#                 lick_vl = np.linalg.norm(licks,axis=-1,ord=2)
#                 lick_vl = 1-np.abs(np.sum(licks*np.exp(phase[np.newaxis,:]),axis=-1))
#                 print(lick_vl)
                if mouse=='4467975.4' and i==0 and sess_ind==0:
                    lick_vl[24:68]=np.nan
                

                if lr_trial[0] == 1:
                    famarm_vl.append(lick_vl[lr_trial==1])
                    novelarm_vl.append(lick_vl[lr_trial==-1])
                else:
                    famarm_vl.append(lick_vl[lr_trial==-1])
                    novelarm_vl.append(lick_vl[lr_trial==1])
                    
                
            
        vl_dict[mouse] = {'fam':famarm_vl,'novel':novelarm_vl}
    return vl_dict
    

In [ ]:
# example mouse

# get sessions 

# get licking behavior
    # licking trial matrix and lick positions timeseries


In [ ]:
# define single session function 


    
    # split by blocks
    
    # correlation within blocks
    
    
    # left vs right correlation in last block
    
    # calculate 

In [ ]:
# licking accuracy effect on novel arm on day 1?

# day to day changes in licking accuracy, last block to first block